**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib
import urllib.request
url = 'https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/refs/heads/master/Assignment4/course_materials/python/validation.py'
urllib.request.urlretrieve(url, 'validation.py')
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 31.7 MB/s eta 0:00:00


In [10]:
from validation import Report

First let's read the RDF file

In [11]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
# Do not change the name of the variables
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.parse(github_storage+"/rdf/data06.ttl", format="TTL")
report = Report()

**TASK 7.1a: For all classes, list each classURI. If the class belogs to another class, then list its superclass.**
**Do the exercise in RDFLib returning a list of Tuples: (class, superclass) called "result". If a class does not have a super class, then return None as the superclass**

In [23]:
# TO DO

result = [] #list of tuples
for i in g.subjects(RDF.type, RDFS.Class):
    superclass = g.value(subject=i, predicate=RDFS.subClassOf)
    result.append((i, superclass if superclass else None))
# Visualize the results
for r in result:
  print(r)

(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Animal'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Student'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#FullProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#InterimAssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'))


In [21]:
## Validation: Do not remove
report.validate_07_1a(result)

TASK 7.1a OK


**TASK 7.1b: Repeat the same exercise in SPARQL, returning the variables ?c (class) and ?sc (superclass)**

In [27]:

query = """
    SELECT ?c ?sc
    WHERE {
        ?c rdf:type rdfs:Class .
        OPTIONAL {
          ?c rdfs:subClassOf ?sc .
          }
    }
    """

for r in g.query(query):
  print(r.c, r.sc)


http://oeg.fi.upm.es/def/people#Person None
http://oeg.fi.upm.es/def/people#Animal None
http://oeg.fi.upm.es/def/people#Professor http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#Student http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#FullProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#AssociateProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#InterimAssociateProfessor http://oeg.fi.upm.es/def/people#AssociateProfessor


In [28]:
## Validation: Do not remove
report.validate_07_1b(query,g)

TASK 7.1b OK


**TASK 7.2a: List all individuals of "Person" with RDFLib (remember the subClasses). Return the individual URIs in a list called "individuals"**


In [29]:
ns = Namespace("http://oeg.fi.upm.es/def/people#")

# variable to return
individuals = []


def get_subclasses(cls):
    subclasses = set()
    for subclass in g.subjects(RDFS.subClassOf, cls):
        subclasses.add(subclass)
        subclasses |= get_subclasses(subclass)  # recursive search
    return subclasses

# Get all subclasses of Person
subclasses = get_subclasses(ns.Person)

# Include Person itself
classes_to_check = subclasses | {ns.Person}

# Find all individuals of Person and its subclasses
for c in classes_to_check:
    for ind in g.subjects(RDF.type, c):
        individuals.append(ind)

# Remove duplicates (just in case)
individuals = list(set(individuals))
# visualize results
for i in individuals:
  print(i)

http://oeg.fi.upm.es/def/people#Raul
http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar


In [30]:
# validation. Do not remove
report.validate_07_02a(individuals)

TASK 7.2a OK


**TASK 7.2b: Repeat the same exercise in SPARQL, returning the individual URIs in a variable ?ind**

In [34]:
query =  """
PREFIX ns: <http://oeg.fi.upm.es/def/people#>


SELECT DISTINCT ?ind
WHERE {
  ?subClass rdfs:subClassOf* ns:Person .
  ?ind a ?subClass .
}
"""

for r in g.query(query):
  print(r.ind)
# Visualize the results

http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar
http://oeg.fi.upm.es/def/people#Raul


In [35]:
## Validation: Do not remove
report.validate_07_02b(g, query)

TASK 7.2b OK


**TASK 7.3:  List the name and type of those who know Rocky (in SPARQL only). Use name and type as variables in the query**

In [36]:
query =  """
    PREFIX ns: <http://oeg.fi.upm.es/def/people#>

    SELECT DISTINCT ?name ?type
    WHERE {
        ?person ns:knows ns:Rocky.
        ?person rdfs:label ?name.
        ?person rdf:type ?type.
    }

    GROUP BY ?name
"""
# TO DO
# Visualize the results
for r in g.query(query):
  print(r.name, r.type)


Asun http://oeg.fi.upm.es/def/people#FullProfessor
Raul http://oeg.fi.upm.es/def/people#InterimAssociateProfessor
Fantasma http://oeg.fi.upm.es/def/people#Animal


In [37]:
## Validation: Do not remove
report.validate_07_03(g, query)

TASK 7.3 OK


**Task 7.4: List the name of those entities who have a colleague with a dog, or that have a collegue who has a colleague who has a dog (in SPARQL). Return the results in a variable called name**

In [41]:
query =  """
    PREFIX people: <http://oeg.fi.upm.es/def/people#>
    SELECT DISTINCT ?name
    WHERE {
        {

            ?person people:hasColleague ?colleague .
            ?colleague people:ownsPet ?perro.
        }
        UNION
        {

            ?person people:hasColleague ?colleague .
            ?colleague people:hasColleague ?colleaguee .
            ?colleaguee people:ownsPet ?perro.
        }
        ?person rdfs:label ?name
  }
"""

for r in g.query(query):
  print(r.name)

# TO DO
# Visualize the results

Asun
Oscar
Raul


In [39]:
## Validation: Do not remove
report.validate_07_04(g,query)
report.save_report("_Task_07")

TASK 7.4 OK
